## crawler class 

In [19]:
import urllib.request
import urllib.parse
from bs4 import BeautifulSoup
import sqlite3
import re

# Create a list of words to ignore
ignorewords={'the':1,'of':1,'to':1,'and':1,'a':1,'in':1,'is':1,'it':1}

class Crawler:
    # Initialize the crawler with the name of database
    def __init__(self, db_name):
        self.con = sqlite3.connect(db_name, timeout=10)
    
    def __del__(self):
        self.con.close()
        
    def db_commit(self):
        self.con.commit()
    
    # Axillary function for getting an entry id and adding
    # it if it's not present
    def get_entry_id(self, table, field, value, create_new=True):
        cur=self.con.execute("select rowid from %s where %s='%s'" % (table,field,value))
        res=cur.fetchone()
        if res==None:
            cur=self.con.execute("insert into %s (%s) values ('%s')" % (table,field,value))
            return cur.lastrowid
        else:
            return res[0] 
    
    # Create index for every page
    def add_to_index(self, url, soup):
        if self.is_indexed(url):
            return 
        print('Indexing %s' % url)
        
        # Get the individual words
        text = self.get_text_only(soup)
        words = self.separate_words(text)
        
        # Get the id of URL
        url_id = self.get_entry_id('urllist', 'url', url)
        
        # Link each word to this url
        for i in range(len(words)):
            word = words[i]
            if word in ignorewords:
                continue
            word_id = self.get_entry_id('wordlist','word',word)
            self.con.execute("insert into wordlocation(urlid,wordid,location) \
                             values (%d,%d,%d)" % (url_id,word_id,i))
        
    # Extract the text from an HTML page (no tags)
    def get_text_only(self, soup):
        v = soup.string
        if v==None:
            c = soup.contents
            result_text=''
            for t in c:
                sub_text = self.get_text_only(t)
                result_text += sub_text + '\n'
            return result_text
        else:
            return v.strip()
    
    # Separate words by any non_whitespace character
    def separate_words(self, text):
        splitter = re.compile('\\W*')
        return [s.lower() for s in splitter.split(text) if s!='']
    
    # Return true if the url is already indexed 
    def is_indexed(self, url):
        u = self.con.execute("select rowid from urllist where url='%s'" % url).fetchone()
        if u != None:
            v = self.con.execute("select * from wordlocation where urlid=%d" % u[0]).fetchone()
            if v != None:
                return True
        return False
    
    # Add a link between two pages
    def add_link_ref(self, urlFrom, urlTo, linkText):
        words = self.separate_words(linkText)
        from_id = self.get_entry_id('urllist', 'url', urlFrom)
        to_id = self.get_entry_id('urllist', 'url', urlTo)
        if from_id == to_id:
            return
        cur = self.con.execute('insert into link(fromid, toid) values (%d, %d)' % (from_id, to_id))
        link_id = cur.lastrowid
        for word in words:
            if word in ignorewords:
                continue
            word_id = self.get_entry_id('wordlist', 'word', word)
            self.con.execute("insert into linkwords(linkid,wordid) values (%d,%d)" % (link_id,word_id))
    
    # Starting with a list of pages, do a breadth 
    # first search to the given depth, indexing pages 
    # as we go
    def crawl(self, pages, depth=2):
        for i in range(depth):
            print('depth %d begins' % i)
            new_pages = set()
            for page in pages:
                try:
                    c = urllib.request.urlopen(page)
                except:
                    print('Could not open %s' % page)
                    continue
                soup = BeautifulSoup(c.read(),'lxml')
                self.add_to_index(page, soup)
                
                links = soup('a')
                for link in links:
                    if ('href' in dict(link.attrs)):
                        url = urllib.parse.urljoin(page, link['href'])
                        if url.find("'")!=-1:
                            continue
                        url=url.split('#')[0]  # remove location portion
                        if url[0:4]=='http' and not self.is_indexed(url):
                            new_pages.add(url)
                        link_text = self.get_text_only(link)
                        self.add_link_ref(page, url, link_text)
                self.db_commit()
            pages = new_pages
    
    # Create the database tables
    def create_index_tables(self):
        self.con.execute('create table urllist(url)')
        self.con.execute('create table wordlist(word)')
        self.con.execute('create table wordlocation(urlid,wordid,location)')
        self.con.execute('create table link(fromid integer,toid integer)')
        self.con.execute('create table linkwords(wordid,linkid)')
        self.con.execute('create index wordidx on wordlist(word)')
        self.con.execute('create index urlidx on urllist(url)')
        self.con.execute('create index wordurlidx on wordlocation(wordid)')
        self.con.execute('create index urltoidx on link(toid)')
        self.con.execute('create index urlfromidx on link(fromid)')
        self.db_commit()
        
    def calculate_pagerank(self, iterations=20):
        # clear out the current PageRank tables
        self.con.execute('drop table if exists pagerank')
        self.con.execute('create table pagerank(urlid primary key, score)')
        
        # Initialize every url with a pagerank of 1
        self.con.execute('insert into pagerank select rowid, 1.0 from urllist')
        self.db_commit()
        
        for i in range(iterations):
            print('Iteration %d' % i)
            for (url_id,) in self.con.execute('select rowid from urllist'):
                pr = 0.15
                
                # Loop through all the pages that link to this one
                for (linker,) in self.con.execute('select distinct fromid from link where toid=%d' % url_id):
                    # Get the pagerank of the linker 
                    linking_pr = self.con.execute('select score from pagerank where urlid=%d' % linker).fetchone()[0]
                    
                    # Get the total number of links from the linker
                    linking_count = self.con.execute('select count(*) from link where fromid=%d' % linker).fetchone()[0]
                    pr += 0.85*(linking_pr/linking_count)
                self.con.execute('update pagerank set score=%f where urlid=%d' % (pr, url_id))
            self.db_commit()

In [20]:
crawler = Crawler('searchindex.db')
# Run create_index_tables when db hasn't been created
crawler.create_index_tables()

In [21]:
page_list = ['http://www.bilibili.com']
crawler.crawl(page_list)

depth 0 begins
Indexing http://www.bilibili.com
{'http://www.bilibili.com/video/ent-handmake-1.html', 'http://www.bilibili.com/video/part-twoelement-1.html', 'http://weibo.com/bilibiliweb', 'http://www.bilibili.com/video/game.html', 'http://www.bilibili.com/video/music-Cover-1.html', 'http://www.bilibili.com/event', 'http://zb.bilibili.com/', 'http://big.bilibili.com/site/big.html', 'http://www.bilibili.com/video/music-perform-1.html', 'http://bml.bilibili.com/index2016/?nav', 'http://www.bilibili.com/video/douga-kichiku-1.html', 'http://live.bilibili.com/i', 'http://www.bilibili.com/video/music-original-1.html', 'http://www.bilibili.com/video/life.html', 'http://space.bilibili.com/', 'http://www.bilibili.com/video/ent-circle-1.html', 'http://activity.bilibili.com', 'https://account.bilibili.com/login', 'http://www.bilibili.com/video/ad-ad-1.html', 'http://bangumi.bilibili.com/anime/timeline', 'http://www.bilibili.com/video/ent-animal-1.html', 'http://www.bilibili.com/video/fashion-inf

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:69: FutureWarning: split() requires a non-empty pattern match.


Indexing http://www.bilibili.com/video/part-twoelement-1.html
Indexing http://weibo.com/bilibiliweb
Indexing http://www.bilibili.com/video/game.html
Indexing http://www.bilibili.com/video/music-Cover-1.html
Indexing http://www.bilibili.com/event
Indexing http://zb.bilibili.com/
Indexing http://big.bilibili.com/site/big.html
Indexing http://www.bilibili.com/video/music-perform-1.html
Indexing http://bml.bilibili.com/index2016/?nav
Indexing http://www.bilibili.com/video/douga-kichiku-1.html
Indexing http://live.bilibili.com/i
Indexing http://www.bilibili.com/video/music-original-1.html
Indexing http://www.bilibili.com/video/life.html
Indexing http://space.bilibili.com/
Indexing http://www.bilibili.com/video/ent-circle-1.html
Indexing http://activity.bilibili.com
Indexing https://account.bilibili.com/login
Indexing http://www.bilibili.com/video/ad-ad-1.html
Indexing http://bangumi.bilibili.com/anime/timeline
Indexing http://www.bilibili.com/video/ent-animal-1.html
Indexing http://www.bili

In [5]:
[row for row in crawler.con.execute('select rowid,urlid from wordlocation where wordid=2')]

[(2, 1),
 (760, 3),
 (1119, 9),
 (1393, 12),
 (1773, 13),
 (8365, 18),
 (8785, 20),
 (9325, 23),
 (9754, 27),
 (9848, 28),
 (10387, 37),
 (10392, 38),
 (12809, 41),
 (13589, 43),
 (13875, 46),
 (14239, 48),
 (14278, 49),
 (16346, 53),
 (16668, 54),
 (18110, 74),
 (18566, 81),
 (24459, 82),
 (24817, 88),
 (25579, 92),
 (27010, 100),
 (28498, 107),
 (29067, 111),
 (29787, 121),
 (29881, 123)]

## Querying 

In [35]:
class Searcher:
    def __init__(self, db_name):
        self.con = sqlite3.connect(db_name)
    
    def __del__(self):
        self.con.close()
        
    def get_match_rows(self, q):
        # Strings to build query
        field_list = 'w0.urlid'
        table_list = ''
        clause_list = ''
        word_ids = []
        
        # Split words by spaces
        words = q.split(' ')
        table_number = 0
        
        for word in words:
            # get id of the word
            word_row = self.con.execute("select rowid from wordlist where word = '%s'" % word).fetchone()
            if word_row != None:
                word_id = word_row[0]
                word_ids.append(word_id)
                if table_number>0:
                    table_list += ','
                    clause_list += ' and '
                    clause_list+='w%d.urlid=w%d.urlid and ' % (table_number-1,table_number)
                field_list+=',w%d.location' % table_number
                table_list+='wordlocation w%d' % table_number      
                clause_list+='w%d.wordid=%d' % (table_number,word_id)
                table_number+=1
                
        # Create the query from the separate parts
        full_query='select %s from %s where %s' % (field_list,table_list,clause_list)
        print (full_query)
        cur=self.con.execute(full_query)
        rows=[row for row in cur]
        
        return rows, word_ids
    
    def get_scored_list(self, rows, word_ids):
        total_scores = dict([(row[0], 0) for row in rows])
        
        # score function
        weighs = [(0.0, self.frequency_score(rows)),
                 (1.0, self.location_score(rows)),
                 (0.0, self.distance_score(rows)),
                 (0.0, self.inbound_link_score(rows)),
                 (1.0, self.page_rank_score(rows))]
        
        for (weight, scores) in weighs:
            for url in total_scores:
                total_scores[url] += weight*scores[url]
                
        return total_scores
    
    def get_url_name(self, id):
        return self.con.execute("select url from urllist where rowid = %d" % id).fetchone()[0]
    
    def query(self, q):
        rows, word_ids = self.get_match_rows(q)
        scores = self.get_scored_list(rows, word_ids)
        ranked_scores = sorted([(score, url) for (url, score) in scores.items()], reverse=1)
        for (score, url_id) in ranked_scores[0:10]:
            print('%f\t%s' % (score, self.get_url_name(url_id)))
            
    def normalize_scores(self, scores, small_is_better=0):
        v_small = 0.00001 # Avoid division by zero errors
        if small_is_better:
            min_score = min(scores.values())
            return dict([(u, float(min_score)/max(v_small,1)) for (u, l) in scores.items()])
        else:
            max_score = max(scores.values())
            if max_score == 0:
                max_score = v_small
            return dict([(u, float(c)/max_score) for (u, c) in scores.items()])
        
    def frequency_score(self, rows):
        counts = dict([(row[0], 0) for row in rows])
        for row in rows:
            counts[row[0]] += 1
        return self.normalize_scores(counts)
    
    def location_score(self, rows):
        locations = dict([(row[0], 0) for row in rows])
        for row in rows:
            loc = sum(row[1:])
            if loc<locations[row[0]]:
                locations[row[0]] = loc
        return self.normalize_scores(locations, small_is_better=1)
    
    def distance_score(self, rows):
        # if there's only one word, everyone wins!
        if len(rows[0])<2:
            return dict([(row[0], 1.0) for row in rows])
        
        # Initial dict with large values
        min_distance = dict([(row[0], 1000000) for row in rows])
        
        for row in rows:
            dist = sum([abs(row[i]-row[i-1]) for i in range(2, len(row))])
            if dist<min_distance[row[0]]:
                min_distance[row[0]] = dist
        return self.normalize_scores(min_distance, small_is_better=1)
    
    def inbound_link_score(self, rows):
        unique_urls = dict([(row[0], 1) for row in rows])
        inbound_count = dict([(u,self.con.execute('select count(*) from link where toid=%d' % u)
                                   .fetchone()[0]) for u in unique_urls])
        return self.normalize_scores(inbound_count) 
    
    def page_rank_score(self,rows):
        pageranks=dict([(row[0],self.con.execute('select score from pagerank where urlid=%d' % row[0]).fetchone()[0]) for row in rows])
        maxrank=max(pageranks.values())
        normalizedscores=dict([(u,float(l)/maxrank) for (u,l) in pageranks.items()])
        return normalizedscores
    
    def linktextscore(self,rows,wordids):
        linkscores=dict([(row[0],0) for row in rows])
        for wordid in wordids:
            cur=self.con.execute('select link.fromid,link.toid from linkwords,link where wordid=%d and linkwords.linkid=link.rowid' % wordid)
        for (fromid,toid) in cur:
            if toid in linkscores:
            pr=self.con.execute('select score from pagerank where urlid=%d' % fromid).fetchone()[0]
            linkscores[toid]+=pr
        maxscore=max(linkscores.values())
        normalizedscores=dict([(u,float(l)/maxscore) for (u,l) in linkscores.items()])
        return normalizedscores

In [36]:
e = Searcher('searchindex.db')
e.query('bilibili 直播')

select w0.urlid,w0.location,w1.location from wordlocation w0,wordlocation w1 where w0.wordid=5 and w0.urlid=w1.urlid and w1.wordid=24
1.000000	http://message.bilibili.com
0.990608	http://app.bilibili.com/
0.983514	http://www.bilibili.com/html/cele.html
0.980617	http://h.bilibili.com/wallpaper?action=list
0.974274	http://www.bilibili.com/html/contact.html
0.972971	http://www.bilibili.com/html/friends-links.html
0.971005	http://www.bilibili.com/html/aboutUs.html
0.952633	https://account.bilibili.com/site/home
0.950351	https://pay.bilibili.com/
0.948912	https://www.bilibili.com/register


## PageRank 

In [23]:
crawler.calculate_pagerank()

Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Iteration 11
Iteration 12
Iteration 13
Iteration 14
Iteration 15
Iteration 16
Iteration 17
Iteration 18
Iteration 19


In [34]:
[row for row in crawler.con.execute('select urllist.url,pagerank.score from pagerank,urllist \
                                    where urllist.rowid=pagerank.urlid order by pagerank.score desc')][0:10]

[('https://passport.bilibili.com/login', 0.384453),
 ('http://www.bilibili.com/html/join-list.html', 0.362684),
 ('https://account.bilibili.com/login?act=exit', 0.275827),
 ('https://account.bilibili.com/answer/addq', 0.264513),
 ('http://live.bilibili.com/i', 0.262633),
 ('http://message.bilibili.com', 0.250216),
 ('http://www.bilibili.com/html/join.html', 0.250216),
 ('http://app.bilibili.com/', 0.247866),
 ('http://www.bilibili.com/video/av120040/', 0.247764),
 ('http://www.bilibili.com/html/cele.html', 0.246091)]